In [ ]:
import sys
import os

parent_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, parent_path)

In [ ]:
from utils.miditool import *

In [ ]:
evts, resolution = get_simplified_event_list("../data/Cras numquam scire.mid")

In [ ]:
evts # event absolute tick, data, event type

In [ ]:
resolution

In [ ]:
notes = convert_to_abs_notes("../data/Cras numquam scire.mid")
notes # note absolute time (microsecond), pitch, absolute duration (microsecond)

In [ ]:
r"""
Now let us see if we can reconstruct the music from `notes`.
I am going to use sythetic wave to reconstruct.
"""

In [ ]:
import numpy as np
from utils.generate import generate_wave

fs = 44100  # sample frequency


def freq(midi_pitch):  # `midi_pitch` is varied in [0, 127]
    return 440 * 2**((midi_pitch - 69) / 12)


def digit_time(microsecond):
    return int(microsecond / 1e6 * fs)


notes = [(digit_time(i.time), freq(i.pitch), digit_time(i.dur)) for i in notes]
totlen = max(i[0]+i[2] for i in notes)
wave = np.zeros(totlen)
for i in notes:
    start = i[0]
    size = i[2]
    wavelet = generate_wave([i[1]], fs, size)
    wave[start:start + size] += wavelet

In [ ]:
import matplotlib.pyplot as plt
plt.plot(wave)
plt.show()

In [ ]:
from scipy.io.wavfile import write
scaled = np.int16(wave/np.max(np.abs(wave)) * 32767)
write('tmp.wav', fs, scaled)

In [ ]:
import IPython
IPython.display.Audio('tmp.wav')

In [ ]:
# original
import music21 as m21
p = m21.converter.parse('../data/Cras numquam scire.mid')
p.show('midi')